In [ ]:
! python train.py --root-imgs /home/jovyan/work/cob-test/pascal-voc/VOC2012 --root-segs /home/jovyan/work/cob-test/trainval --run-path /home/jovyan/work/cob-test/runs/cob --cuda

In [ ]:
import scipy.io as io
lbl_path = "trainval/2008_000002.mat"
data = io.loadmat(lbl_path)
lbl = data['LabelMap']

In [ ]:
lbl

In [ ]:
import numpy as np
from utils.dataloader import interpolate_to_polygon

In [ ]:
image = interpolate_to_polygon(lbl).astype('float64')
image *= 255.0/image.max()

In [ ]:
from PIL import Image
img = Image.fromarray(np.uint8(image * 255) , 'L')

In [ ]:
img

In [ ]:
interpolate_to_polygon(lbl).astype(np.uint8)

In [1]:
import torch
from torch.utils.data import DataLoader
from utils.dataloader import CobDataLoader

In [ ]:
dset_val = CobDataLoader(root_imgs="/home/jovyan/work/cob-test/pascal-voc/VOC2012", root_segs="/home/jovyan/work/cob-test/trainval", split='val')
dl_val = DataLoader(dset_val, collate_fn=dset_val.collate_fn, batch_size=1)

In [ ]:
from models.cobnet import CobNet
model = CobNet()
model.load_state_dict(torch.load("runs/cob/checkpoints/cp_or.pth.tar"))

In [ ]:
batch = next(iter(dl_val))
batch = batch["image"]

In [ ]:
from skimage.io import imread
from imgaug import augmenters as iaa
from utils.augmenters import Normalize, rescale_augmenter

normalization_mean=[0.485, 0.456, 0.406]
normalization_std=[0.229, 0.224, 0.225]

aug = iaa.Sequential([
    iaa.size.Resize(250),
    iaa.Flipud(p=0.5),
    iaa.Fliplr(p=.5),
    iaa.Fliplr(p=.5),
    iaa.Rotate([360 / 4 * i for i in range(4)]),
    rescale_augmenter,
    Normalize(mean=normalization_mean, std=normalization_std)
])
aug_det = aug.to_deterministic()

In [ ]:
image = imread("pascal-voc/VOC2012/JPEGImages/2007_000033.jpg")
image = image[np.newaxis, ...]
#image_t = np.stack((image[:,:,0], image[:,:,1], image[:,:,2]), axis = 0)
image2 = imread("denmark/1.jpg")
image2 = image2[np.newaxis, ...]
#image2_t = np.stack((image2[:,:,0], image2[:,:,1], image2[:,:,2]), axis = 0)

sample = aug_det(images=image2)

In [ ]:
csample = sample[0]
in_img = np.stack((csample[:,:,0], csample[:,:,1], csample[:,:,2]), axis = 0)
in_img = in_img[np.newaxis, ...]
model.eval()
with torch.no_grad():
    res = model(torch.tensor(in_img).float())

In [ ]:
from torchvision.utils import make_grid, save_image
n_orient=4

fine = res['y_fine'].sigmoid()
coarse = res['y_coarse'].sigmoid()

total_orients = len(res['orientations'])
orients_idx = np.arange(0, total_orients, step=total_orients // n_orient)
orients = []
for i in orients_idx:
    o_ = make_grid(res['orientations'][i].sigmoid(), nrow=1)
    orients.append(o_)

save_image(fine, 'test.png')
save_image(coarse, 'coarse.png')
save_image([*orients], 'orients.png')

In [ ]:
img = Image.fromarray(image2[0] , 'RGB')
img

In [2]:
torch.cuda.is_available()

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


False